<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка
### Иморт бибилиотек

In [92]:
import numpy as np
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, Pool

from pymystem3 import Mystem

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')


In [93]:
#from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline

#model_path = "martin-ha/toxic-comment-model"
#tokenizer = AutoTokenizer.from_pretrained(model_path)
##model = AutoModelForSequenceClassification.from_pretrained(model_path)

#pipeline =  TextClassificationPipeline(model=model, tokenizer=tokenizer)
#print(pipeline('This is a test text.'))

### Загрузка данных

In [94]:
try:
    df = pd.read_csv('./datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Текст на английском, поэтому будем исопльзовать обычный BERT.  

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Типы данных в порядке. В датасете около 160 тыс. фраз.

In [96]:
df.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Целевой столбец - булевый, что нам и требовалось.
Доля токсичных комментариев - чуть больше 10%  
Не понятно, что за столбец unnamed, похоже на дублирование индекса, проверим.

Все верно, это дубликат, удалим его.

In [97]:
df = df[['text','toxic']]
df.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Уменьшим датафрейм до 20000. При необходимости увеличим позже.

In [98]:
#df = df.sample(80000,random_state=12345)
df['toxic'].mean()

0.10161213369158527

### Обработка текста и деление

#### Деление на тренировочную и тестовую выборки

In [99]:
train = df.sample(frac=0.8, random_state=42)
test = df.drop(train.index)


In [100]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\atropinskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [101]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

In [102]:
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

train['lemm_text'] = train['text'].apply(cleaning)
test['lemm_text'] = test['text'].apply(cleaning)

y_train = train['toxic']
y_test = test['toxic']

train.head()

,text,toxic,lemm_text
31015,"Sometime back, I just happened to log on to ww...",0,sometime back i just happened to log on to www...
102832,"""\n\nThe latest edit is much better, don't mak...",0,the latest edit is much better dont make this ...
67317,""" October 2007 (UTC)\n\nI would think you'd be...",0,october utc i would think youd be able to ge...
81091,Thanks for the tip on the currency translation...,0,thanks for the tip on the currency translation...
90091,I would argue that if content on the Con in co...,0,i would argue that if content on the con in co...


In [103]:
tfidf_train = count_tf_idf.fit_transform(train['lemm_text'])
tfidf_test = count_tf_idf.transform(test['lemm_text'])

## Обучение

#### Random forest

In [104]:
model_rf = RandomForestClassifier(random_state=12345)

params_rf = {
    'n_estimators': [13],
    'max_depth':[5],
    'max_features' : [10000]
}

grid = GridSearchCV(model_rf, param_grid=params_rf, scoring='f1', cv=3, verbose=True, n_jobs=-1)

best_grid = grid.fit(tfidf_train, y_train)
print('Best parameters is:', grid.best_params_)
print('Best score for RF is:', grid.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters is: {'max_depth': 5, 'max_features': 10000, 'n_estimators': 13}
Best score for RF is: 0.24186320367318845


#### Logistic regression

In [105]:
model_lr = LogisticRegression(random_state=42, penalty = 'l1', solver='liblinear', max_iter=200)
model_lr.fit(tfidf_train, y_train)
valid_pred = model_lr.predict(tfidf_train)
print ('Best score for LR is:',f1_score(y_train, valid_pred))

Best score for LR is: 0.7861660253858742


#### Catboost

In [106]:
catboost = CatBoostClassifier(logging_level = 'Silent',
                              iterations=300,
                              learning_rate=0.5,
                              l2_leaf_reg=1)

catboost.fit(tfidf_train,y_train) 
pred_cb = catboost.predict(tfidf_train)
print ('Best f1 score for CB is:',f1_score(y_train, pred_cb))

Best f1 score for CB is: 0.841803067957837


#### Тестирование

In [107]:
test_pred = catboost.predict(tfidf_test)
print ('Best f1 score for CB is:',f1_score(y_test, test_pred))
test_pred = model_lr.predict(tfidf_test)
print ('Best f1 score for LR is:',f1_score(y_test, test_pred))

Best f1 score for CB is: 0.7695264721300018
Best f1 score for LR is: 0.780411293133496


## Выводы

1. В данном проекте мы преобразовали текст, токенизировали его, удалили стоп-слов, что позволило нам построить несколько моделей.
2. Лучше всех отработала логистическая регрессия, при высокой скорости обучения менее секунды - она показала значение f1 78
3. Также неплохо отработал catboost, но обучался 4 минуты.
4. Модель RF отработала хуже. всех.

В результате предлагаю использовать модель Логистической регрессии, которая показала лучший результат как по скорости, так и по точности.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны